In [94]:
from elasticsearch import Elasticsearch
import numpy
import json
import pandas as pd

# Loading the json

In [106]:
df = pd.read_json(r'data/data2/400gbJson.json')

In [107]:
df.head()

,name,aliases,genres,description,type,release_date
0,Jail Breakers,[Dasso yugi@en],"[Action Film, Crime Fiction Film]",[Jail Breakers is a crime fiction film directe...,"[common.topic, film.film]",1976-06-19
1,Kvelden er din,[None],[None],[None],"[common.topic, tv.tv_program]",2012-03-24
2,Grandeur et decadence dun petit commerce de Ci...,[None],[None],[Grandeur et decadence dun petit commerce de C...,"[common.topic, film.film]",1986
3,The Doors: Live in Europe (1968),[None],"[Music Film, Documentary Film]",[None],"[common.topic, film.film, media_common.netflix...",1991
4,Smert Tairova,[None],[None],[Smert Tairova is a 2003 film directed by Bori...,"[award.award_nominated_work, award.award_winni...",2003


In [108]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339674 entries, 0 to 339673
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   name          339674 non-null  object
 1   aliases       339674 non-null  object
 2   genres        339674 non-null  object
 3   description   339674 non-null  object
 4   type          339674 non-null  object
 5   release_date  339674 non-null  object
dtypes: object(6)
memory usage: 15.5+ MB


Num of records:

In [109]:
len(df)

339674

In [167]:
df.columns

Index(['name', 'aliases', 'genres', 'description', 'type', 'release_date'], dtype='object')

### aliases

In [ ]:
df['aliases'].value_counts()

### genres

### type

### release_date

Lets see the oldest movie in our dataset:

In [110]:
df[df['release_date'] == df['release_date'].min()]

,name,aliases,genres,description,type,release_date
156470,The Downtime,[None],"[Drama Film, Short Film TV Program, Short Film...",[The Downtime is a short comedy drama film dir...,"[common.topic, film.film]",0214-02-28


It seems that there are some inconsistent data from the freebase since the first recorded film was produced in the end of the 19th century. We adjust the data so that we only view data that are past this from and past this date.

In [111]:
date_column = df[(df['release_date'] >= '1870') & (df['release_date'] <= "2020") & (df['release_date'] != "None")]['release_date']

In [112]:
date_column.min()

'1880'

In [156]:
date_column.max()

'2019'

In [157]:
date_column

0         1976
1         2012
2         1986
3         1991
4         2003
          ... 
339648    2013
339653    2013
339658    2013
339661    2013
339673    1976
Name: release_date, Length: 296369, dtype: object

In [158]:
date_column = pd.to_datetime(date_column)

In [159]:
date_column = date_column.map(lambda x: x.strftime('%Y'))

In [160]:
print(date_column)

0         1976
1         2012
2         1986
3         1991
4         2003
          ... 
339648    2013
339653    2013
339658    2013
339661    2013
339673    1976
Name: release_date, Length: 296369, dtype: object


In [161]:
date_column.describe()

count     296369
unique       133
top         2013
freq       15842
Name: release_date, dtype: object

In [162]:
date_column.value_counts()

2013    15842
2012    14253
2008    14068
2009    13274
2011    12814
        ...  
1890        2
1889        2
1888        1
1880        1
1893        1
Name: release_date, Length: 133, dtype: int64

In [164]:
date_column.value_counts()[:40].to_frame() 

,release_date
2013,15842
2012,14253
2008,14068
2009,13274
2011,12814
2010,12487
2007,11653
2006,11552
2005,10541
2004,9707


In [155]:
date_column.value_counts()[40:80].to_frame()

,release_date
1971,1805
1974,1762
1973,1738
1968,1650
1975,1650
1970,1643
1969,1610
1965,1572
1967,1560
1966,1485


In [150]:
date_column.value_counts()[80:].to_frame()

,release_date
1933,735
1931,728
1932,704
1944,693
1945,676
1930,644
1916,598
1915,590
1929,578
1928,553


In [7]:
es = Elasticsearch(HOST="http://localhost", PORT=9200)
es = Elasticsearch()

In [9]:
es.indices.create(index="second_index")

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'second_index'}

In [10]:
es.indices.exists("first_index")

True